In [1]:
!pip install torch_geometric

     |████████████████████████████████| 133kB 2.6MB/s 
     |████████████████████████████████| 348kB 7.5MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.4.3-cp36-none-any.whl size=234873 sha256=61c82bfa827cbf9ab444e07230868a239bcd0f7d593761016b1e4c5d5962aa1b
  Stored in directory: /root/.cache/pip/wheels/e2/c1/09/8693feee3f97e440d68b09abfca8b4c1e97150ace350b5003f
  Created wheel for plyfile: filename=plyfile-0.7.1-cp36-none-any.whl size=32828 sha256=ec7c196269ca04e278236296705a94f2b8d71f5ef6fa450f135c47733435d1cd
  Stored in directory: /root/.cache/pip/wheels/d6/0d/bf/6d603d81b98604d2ecfd5e99d4ab7c9af664fd5285ab82bbb0
Successfully built torch-geometric plyfile


In [2]:
!pip install torch-scatter==latest+cu101 torch-sparse==latest+cu101 -f https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.4.0.html

Looking in links: https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.4.0.html
     |████████████████████████████████| 10.6MB 703kB/s 
     |████████████████████████████████| 11.5MB 366kB/s 


In [0]:
import sys
sys.path.insert(0, '/content/drive/My Drive')

In [0]:
import torch
import numpy as np
import networkx as nx
import torch.nn as nn
import time

In [0]:
from utils import EarlyStopping

In [0]:
%pwd

'/content'

In [0]:
from torch_geometric.datasets import Planetoid

In [0]:
Cora = Planetoid(root='/content/drive/My Drive/', name='Cora')
cora = Cora[0]

In [0]:
C_S = Planetoid(root='/content/drive/My Drive/', name='CiteSeer')
cite_seer = C_S[0]

In [0]:
PM = Planetoid(root='/content/drive/My Drive/', name='PubMed')
pub_med = PM[0]

In [0]:
def Rand_sw(G, n=1):
    #np.random.seed(8848)
    A = nx.to_numpy_array(G, nodelist=range(len(G)))
    w = np.random.random(A.shape)
    A_upper = np.triu(A)*w
    rw = A_upper + A_upper.T + n*np.eye(A.shape[0])
    return rw

In [0]:
def Rand_aw(G, n=1):
    #np.random.seed(1024)
    A = nx.to_numpy_array(G, nodelist=range(len(G))) 
    w = np.random.random(A.shape)
    rw = A*w + n*np.eye(A.shape[0])
    return rw

In [0]:
data = cite_seer
edge_index, _ = remove_self_loops(data.edge_index)
edge_index, _ = add_self_loops(edge_index, num_nodes=data.num_nodes)
edges = edge_index.numpy().T

rg = nx.Graph()

rg.add_edges_from(edges)

In [0]:
num_features = data.num_features
num_classes = len(set(data.y.numpy()))

In [0]:
import torch
import torch.nn as nn
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.utils import remove_self_loops, add_self_loops
from torch_geometric.nn.conv import MessagePassing


class rw_GNN(MessagePassing):
    def __init__(self, in_channels, out_channels, rw, k=3, cached=True, bias=True,
                 **kwargs):
        
        super(rw_GNN, self).__init__(aggr='add', **kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.rw = rw
        self.k = k
        self.cached = cached
        self.lin = Linear(in_channels, out_channels, bias=True)

        self.reset_parameters()

    def reset_parameters(self):
        gain = nn.init.calculate_gain('relu')
        nn.init.xavier_uniform_(self.lin.weight, gain=gain)
        nn.init.zeros_(self.lin.bias)
        #self.lin.reset_parameters()
        self.cached_result = None
        self.cached_num_edges = None
        
    '''def reset_parameters(self):
        self.lin.reset_parameters()
        self.cached_result = None
        self.cached_num_edges = None'''

    def forward(self, x, edge_index):
        """"""
        if self.cached and self.cached_result is not None:
            if edge_index.size(1) != self.cached_num_edges:
                raise RuntimeError(
                    'Cached {} number of edges, but found {}. Please '
                    'disable the caching behavior of this layer by removing '
                    'the `cached=True` argument in its constructor.'.format(
                        self.cached_num_edges, edge_index.size(1)))

      

        if not self.cached or self.cached_result is None:
            self.cached_num_edges = edge_index.size(1)
            #edge_index 处理好
            #edge_index, _ = remove_self_loops(edge_index)
            #edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
            #权值对齐
            #rw的索引array与tensor均可
            norm = self.rw[edge_index[0], edge_index[1]]
            
            #norm = self.rw[edge_index[1], edge_index[0]]
            
            if not torch.is_tensor(norm):
                norm = torch.tensor(norm, dtype=x.dtype,
                                     device=edge_index.device)
            
            x = self.lin(x)
            
            
            for k in range(self.k):
                x = self.propagate(edge_index, x=x, norm=norm) 
            #self.cached_result = x
       
        '''if self.cached:
            x = self.lin(self.cached_result)'''
        return x
    
    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j

    def __repr__(self):
        return '{}({}, {}, K={})'.format(self.__class__.__name__,
                                         self.in_channels, self.out_channels,
                                         self.k)

In [0]:
sw = Rand_sw(rg, n=1)

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = rw_GNN(
            num_features, num_classes, rw=sw , k=2, cached=True)

    def forward(self):
        x = self.conv(data.x, data.edge_index)
        return F.log_softmax(x, dim=1)

In [0]:
das = []
for _ in range(10):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model, data = Net().to(device), data.to(device)

  #cora_pytorch
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.02, weight_decay=5e-4)
  #pubmed
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-3)
  #cite_seer,
  optimizer = torch.optim.Adam(model.parameters(), lr=0.2, weight_decay=5e-2)
  

  def train():
      model.train()
      optimizer.zero_grad()
      loss = F.nll_loss(model()[data.train_mask], data.y[data.train_mask])
      loss.backward()
      optimizer.step()
      return loss

  def test(mask):
      model.eval()
      with torch.no_grad():
          logits = model()
          pred = logits[mask].max(1)[1]
          acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
          
      return acc


  early_stop = True
  if early_stop:
      stopper = EarlyStopping(patience=100)
  dur = []
  print(model)

  for epoch in range(1, 101):
      model.train()
      if epoch >= 3:
          t0 = time.time()
      loss = train()
      
      if epoch >= 3:
          dur.append(time.time() - t0)
      
      val_acc = test(data.val_mask)
      
      if early_stop:
          if stopper.step(val_acc, model):   
              break
      
      '''print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
                " ValAcc {:.4f}".
                format(epoch, np.mean(dur), loss, val_acc))'''
      
  print() # 空出一行
      
  if early_stop:
      model.load_state_dict(torch.load('es_checkpoint.pt'))
  test_acc = test(data.test_mask)
  das.append(test_acc)
  print("Test Accuracy {:.4f}".format(test_acc))

Net(
  (conv): rw_GNN(3703, 6, K=2)
)

Test Accuracy 0.6210
Net(
  (conv): rw_GNN(3703, 6, K=2)
)

Test Accuracy 0.6340
Net(
  (conv): rw_GNN(3703, 6, K=2)
)

Test Accuracy 0.6290
Net(
  (conv): rw_GNN(3703, 6, K=2)
)

Test Accuracy 0.6260
Net(
  (conv): rw_GNN(3703, 6, K=2)
)

Test Accuracy 0.6330
Net(
  (conv): rw_GNN(3703, 6, K=2)
)

Test Accuracy 0.6040
Net(
  (conv): rw_GNN(3703, 6, K=2)
)

Test Accuracy 0.6270
Net(
  (conv): rw_GNN(3703, 6, K=2)
)

Test Accuracy 0.6390
Net(
  (conv): rw_GNN(3703, 6, K=2)
)

Test Accuracy 0.6220
Net(
  (conv): rw_GNN(3703, 6, K=2)
)

Test Accuracy 0.6350


In [0]:
das

[0.621, 0.634, 0.629, 0.626, 0.633, 0.604, 0.627, 0.639, 0.622, 0.635]

In [0]:
np.mean(das)

0.627

In [0]:
#pubmed
k=2
lr=0.05, weight_decay=5e-4 0.7590   0.7490
lr=0.5, weight_decay=5e-4  0.7530.  0.7610
lr=0.02, weight_decay=5e-3
k=5



In [0]:
 #citeseer
 lr=0.2, weight_decay=5e-2  0.6600  0.6340
  lr=0.1, weight_decay=5e-2 0.6540 0.6470
    lr=0.5, weight_decay=5e-2 0.628 0.57
  lr=0.02, weight_decay=5e-2 0.625 0.648
lr=0.01, weight_decay=5e-2 0.644. 0.643


lr=0.2, weight_decay=5e-1 0.5390 0.517

  lr=0.2, weight_decay=5e-3 0.6470  0.63
 lr=0.2, weight_decay=5e-3  0.6130. 0.6120
lr=0.02, weight_decay=5e-4 0.59, 0.57
lr=0.2, weight_decay=5e-4 0.6070 0.6230



In [0]:
 cora
lr=0.05, weight_decay=5e-8, 0.759
lr=0.05, weight_decay=5e-10, 0.743
lr=0.02, weight_decay=5e-8, 0.7780
lr=0.01, weight_decay=5e-8, 0.7350

lr=0.03, weight_decay=5e-8 0.7930**

lr=0.003, weight_decay=5e-8 0.7350

lr=0.02, weight_decay=5e-4, 0.7670
lr=0.05, weight_decay=5e-4,0.7260

lr=0.05, weight_decay=5e-6, 0.7530


In [0]:
data = pub_med

In [0]:
num_features = data.num_features
num_classes = len(set(data.y.numpy()))

In [28]:
data

Data(edge_index=[2, 88648], test_mask=[19717], train_mask=[19717], val_mask=[19717], x=[19717, 500], y=[19717])

## r_GAT

In [0]:
import torch
from torch.nn import Parameter
import torch.nn.functional as F
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax
import numpy as np
import torch.nn as nn


class R_GATConv(MessagePassing):

    def __init__(self, in_channels, out_channels, seed=521,k=2, bias=True, negative_slope=0.2, dropout=0.6):
        super(R_GATConv, self).__init__(aggr='add')

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.k = k
        self.negative_slope = negative_slope
        self.dropout = dropout
        self.bias = bias

        #self.weight = Parameter(
            ##torch.Tensor(in_channels, out_channels))
        
        
        self.lin = nn.Linear(in_channels, out_channels, bias=bias)
        np.random.seed(seed)
        self.att = torch.tensor(np.random.random((1, 2*out_channels)))
        
        self.reset_parameters()

    '''def reset_parameters(self):
        gain = nn.init.calculate_gain('relu')
        nn.init.xavier_normal_(self.lin.weight, gain=gain)
        nn.init.xavier_normal_(self.att, gain=gain)
        if self.bias:
            nn.init.zeros_(self.lin.bias)'''
    def reset_parameters(self):
        self.lin.reset_parameters()
    
    def My_norms(self, x, edge_index, num_nodes):
        edge_index_j = edge_index[0]
        edge_index_i = edge_index[1]
        x_j = x[edge_index_j]
        x_i = x[edge_index_i]
        #print('x_i', x_i.shape)
        ''' alpha0 = (torch.cat([x_i, x_j], dim=-1) * self.att).sum(dim=-1)
        alpha1 = (torch.cat([x_j, x_i], dim=-1) * self.att).sum(dim=-1)
        
        alpha = (alpha0 + alpha1)/2.0'''
        
        alpha = (torch.cat([x_i, x_j], dim=-1) * self.att).sum(dim=-1)
        #print('al', alpha.shape)
        self_weight = edge_index_i== edge_index_j
        alpha[self_weight] += 1.0
        alpha = F.leaky_relu(alpha, self.negative_slope)
        alpha = softmax(alpha, edge_index_i, num_nodes)
        
        # Sample attention coefficients stochastically.
        alpha = F.dropout(alpha, p=self.dropout, training=self.training)
        
        return alpha
    
    
    def forward(self, x, edge_index, size=None):
        """"""
        if size is None and torch.is_tensor(x):
            edge_index, _ = remove_self_loops(edge_index)
            edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        
        x = self.lin(x)
        
        norm = self.My_norms(x, edge_index, x.size(0))
            
        for _ in range(self.k):
            x =  self.propagate(edge_index, size=size, x=x, norm=norm)
            
        return x


    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j
    
    def __repr__(self):
        return '{}({}, {}, heads={})'.format(self.__class__.__name__,
                                             self.in_channels,
                                             self.out_channels, self.k)

In [0]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = R_GATConv(
            num_features, num_classes,seed=1314,k=3)

    def forward(self):
        x, edge_index = data.x, data.edge_index
        x = self.conv(x, edge_index)
        return F.log_softmax(x, dim=1)

In [31]:
das = []
for _ in range(10):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model, data = Net().to(device), data.to(device)

  #cora_pytorch
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.02, weight_decay=5e-4)
  #pubmed
  optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-3)
  #cite_seer,
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.2, weight_decay=5e-2)
  

  def train():
      model.train()
      optimizer.zero_grad()
      loss = F.nll_loss(model()[data.train_mask], data.y[data.train_mask])
      loss.backward()
      optimizer.step()
      return loss

  def test(mask):
      model.eval()
      with torch.no_grad():
          logits = model()
          pred = logits[mask].max(1)[1]
          acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
          
      return acc


  early_stop = True
  if early_stop:
      stopper = EarlyStopping(patience=100)
  dur = []
  print(model)

  for epoch in range(1, 101):
      model.train()
      if epoch >= 3:
          t0 = time.time()
      loss = train()
      
      if epoch >= 3:
          dur.append(time.time() - t0)
      
      val_acc = test(data.val_mask)
      
      if early_stop:
          if stopper.step(val_acc, model):   
              break
      
      '''print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
                " ValAcc {:.4f}".
                format(epoch, np.mean(dur), loss, val_acc))'''
      
  print() 
      
  if early_stop:
      model.load_state_dict(torch.load('es_checkpoint.pt'))
  test_acc = test(data.test_mask)
  das.append(test_acc)
  print("Test Accuracy {:.4f}".format(test_acc))

Net(
  (conv): R_GATConv(500, 3, heads=3)
)

Test Accuracy 0.7630
Net(
  (conv): R_GATConv(500, 3, heads=3)
)

Test Accuracy 0.7880
Net(
  (conv): R_GATConv(500, 3, heads=3)
)

Test Accuracy 0.7750
Net(
  (conv): R_GATConv(500, 3, heads=3)
)

Test Accuracy 0.7810
Net(
  (conv): R_GATConv(500, 3, heads=3)
)

Test Accuracy 0.7770
Net(
  (conv): R_GATConv(500, 3, heads=3)
)

Test Accuracy 0.7750
Net(
  (conv): R_GATConv(500, 3, heads=3)
)

Test Accuracy 0.7730
Net(
  (conv): R_GATConv(500, 3, heads=3)
)

Test Accuracy 0.7740
Net(
  (conv): R_GATConv(500, 3, heads=3)
)

Test Accuracy 0.7650
Net(
  (conv): R_GATConv(500, 3, heads=3)
)

Test Accuracy 0.7780


In [32]:
das

[0.763, 0.788, 0.775, 0.781, 0.777, 0.775, 0.773, 0.774, 0.765, 0.778]

In [33]:
np.mean(das)

0.7749

In [34]:
print('done!!')

done!!


In [0]:
lr=0.02, weight_decay=5e-4 k=3 **
0.8080. 0.8070
  lr=0.01, weight_decay=5e-4 k=3
0.8160. 0.8030

lr=0.05, weight_decay=5e-4 k=3**



lr=0.02, weight_decay=5e-4 k=5
0.7940. 0.8000
lr=0.02, weight_decay=5e-4 k=2
Test Accuracy 0.7960
0.8050

lr=0.02, weight_decay=5e-2
0.7960.  0.8050


